In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [3]:
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df['Outcome']

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [7]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [8]:
X.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [9]:
y.shape

(768,)

In [29]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.283, random_state=42)

In [30]:
X_train.shape

(550, 8)

In [31]:
X_test.shape

(218, 8)

In [39]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [66]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the MLP model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define the learning rate reduction callback
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',   # Monitor validation loss for learning rate reduction
    factor=0.1,            # Reduce learning rate by a factor of 0.1
    patience=5,            # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6            # Minimum learning rate
)
# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)
# Train the model
model.fit(X_train, y_train,epochs=20, batch_size=32,callbacks=[lr_callback,checkpoint_callback,early_stopping_callback])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print(loss, accuracy )

Epoch 1/20
18/18 [==============================] - 1s 4ms/step - loss: 0.6681 - accuracy: 0.6400 - lr: 0.0010
Epoch 2/20
18/18 [==============================] - 0s 3ms/step - loss: 0.6082 - accuracy: 0.6818 - lr: 0.0010
Epoch 3/20
18/18 [==============================] - 0s 3ms/step - loss: 0.5462 - accuracy: 0.7327 - lr: 0.0010
Epoch 4/20
18/18 [==============================] - 0s 2ms/step - loss: 0.5119 - accuracy: 0.7582 - lr: 0.0010
Epoch 5/20
18/18 [==============================] - 0s 3ms/step - loss: 0.5021 - accuracy: 0.7418 - lr: 0.0010
Epoch 6/20
18/18 [==============================] - 0s 3ms/step - loss: 0.4868 - accuracy: 0.7545 - lr: 0.0010
Epoch 7/20
18/18 [==============================] - 0s 3ms/step - loss: 0.4861 - accuracy: 0.7764 - lr: 0.0010
Epoch 8/20
18/18 [==============================] - 0s 3ms/step - loss: 0.4832 - accuracy: 0.7745 - lr: 0.0010
Epoch 9/20
18/18 [==============================] - 0s 3ms/step - loss: 0.4848 - accuracy: 0.7636 - lr: 0.0010
E